In [1]:
import nltk
from nltk.tokenize import sent_tokenize, word_tokenize
import random
from nltk.corpus import movie_reviews

In [2]:
from nltk.classify.scikitlearn import SklearnClassifier
from sklearn.naive_bayes import MultinomialNB, GaussianNB, BernoulliNB
import pickle

In [3]:
from sklearn.linear_model import LogisticRegression, SGDClassifier
from sklearn.svm import SVC, LinearSVC, NuSVC

In [4]:
from nltk.classify import ClassifierI
from statistics import mode

In [5]:
class VoteClassifier(ClassifierI):
    def __init__(self, *classifiers):
        self._classifiers = classifiers
        
    def classify(self, features):
        votes = []
        for c in self._classifiers:
            v = c.classify(features)
            votes.append(v)
        return mode(votes)
    
    def confidence(self, features):
        votes = []
        for c in self._classifiers:
            v = c.classify(features)
            votes.append(v)
        
        choice_votes = votes.count(mode(votes))
        conf = choice_votes / len(votes)
        return conf

In [6]:
short_pos = open("short_reviews/positive.txt","r").read()
short_neg = open("short_reviews/negative.txt","r").read()

In [7]:
documents = []
all_words = []

In [8]:
#  ----------------------------- Old Code--------------------------------

# for r in short_pos.split('\n'):
#     documents.append((r,"pos"))
# for r in short_neg.split('\n'):
#     documents.append((r,"neg"))

# for w in short_pos_words:
#     all_words.append(w.lower())    
# for w in short_neg_words:
#     all_words.append(w.lower())

# -------------------------------- END -----------------------------------

In [9]:
short_pos_words = word_tokenize(short_pos)
short_neg_words = word_tokenize(short_neg)

In [10]:
#  j is adject, r is adverb, and v is verb
#allowed_word_types = ["J","R","V"]
allowed_word_types = ["J"]

In [11]:
#  ------------------- Replacement of Old Code ----------------

for p in short_pos.split('\n'):
    documents.append( (p, "pos") )
    words = word_tokenize(p)
    pos = nltk.pos_tag(words)
    for w in pos:
        if w[1][0] in allowed_word_types:
            all_words.append(w[0].lower())

In [12]:
#  ------------------- Replacement of Old Code ----------------

for p in short_neg.split('\n'):
    documents.append( (p, "neg") )
    words = word_tokenize(p)
    pos = nltk.pos_tag(words)
    for w in pos:
        if w[1][0] in allowed_word_types:
            all_words.append(w[0].lower())

In [13]:
save_documents = open("pickled_algos/documents.pickle","wb")
pickle.dump(documents, save_documents)
save_documents.close()

In [14]:
all_words = nltk.FreqDist(all_words)

In [15]:
word_features = list(all_words.keys())[:5000]

In [16]:
save_word_features = open("pickled_algos/word_features5k.pickle","wb")
pickle.dump(word_features, save_word_features)
save_word_features.close()

In [17]:
def find_features(documents):
    words = word_tokenize(documents)
    features = {}
    for w in word_features:
        features[w] = (w in words)
    return features

In [18]:
featuresets = [(find_features(rev), category) for (rev, category) in documents]

In [45]:
save_features = open("pickled_algos/featuresets.pickle","wb")
pickle.dump(featuresets, save_features)
save_features.close()

In [19]:
random.shuffle(featuresets)

In [20]:
training_set = featuresets[:10000]
testing_set = featuresets[10000:]

# training_set = featuresets[100:]
# testing_set = featuresets[:100]

In [21]:
classifier = nltk.NaiveBayesClassifier.train(training_set)
# classifier_f = open("naivebayes.pickle","rb")
# classifier = pickle.load(classifier_f)
# classifier_f.close()

In [22]:
print("Naiye Bayes Algo Accuracy Percent:", (nltk.classify.accuracy(classifier, testing_set))*100)
classifier.show_most_informative_features(15)

Naiye Bayes Algo Accuracy Percent: 73.94578313253012
Most Informative Features
              engrossing = True              pos : neg    =     19.7 : 1.0
                  boring = True              neg : pos    =     19.3 : 1.0
                 generic = True              neg : pos    =     16.9 : 1.0
                 routine = True              neg : pos    =     16.3 : 1.0
               inventive = True              pos : neg    =     15.7 : 1.0
                mediocre = True              neg : pos    =     14.9 : 1.0
                    flat = True              neg : pos    =     14.9 : 1.0
              refreshing = True              pos : neg    =     14.4 : 1.0
                    warm = True              pos : neg    =     12.7 : 1.0
                    dull = True              neg : pos    =     12.6 : 1.0
               wonderful = True              pos : neg    =     12.2 : 1.0
                    lame = True              neg : pos    =     11.6 : 1.0
               realis

In [23]:
save_classifier = open("pickled_algos/originalnaivebayes5k.pickle","wb")
pickle.dump(classifier, save_classifier)
save_classifier.close()

In [24]:
MNB_classifier = SklearnClassifier(MultinomialNB())
MNB_classifier.train(training_set)
print("MNB_Classifier Accuracy:", (nltk.classify.accuracy(MNB_classifier,testing_set))*100)

MNB_Classifier Accuracy: 74.24698795180723


In [25]:
save_classifier = open("pickled_algos/MNB_classifier5k.pickle","wb")
pickle.dump(MNB_classifier, save_classifier)
save_classifier.close()

In [26]:
BernouliNB_classifier = SklearnClassifier(BernoulliNB())
BernouliNB_classifier.train(training_set)
print("GaussianNB_Classifier Accuracy:", (nltk.classify.accuracy(BernouliNB_classifier, testing_set))*100)

GaussianNB_Classifier Accuracy: 74.84939759036145


In [27]:
save_classifier = open("pickled_algos/BernouliNB_classifier5k.pickle","wb")
pickle.dump(BernouliNB_classifier, save_classifier)
save_classifier.close()

In [28]:
LogisticRegression_classifier = SklearnClassifier(LogisticRegression())
LogisticRegression_classifier.train(training_set)
print("LogisticRegression_classifier accuracy percent:", (nltk.classify.accuracy(LogisticRegression_classifier, testing_set))*100)

LogisticRegression_classifier accuracy percent: 72.28915662650603


In [29]:
save_classifier = open("pickled_algos/LogisticRegression_classifier5k.pickle","wb")
pickle.dump(LogisticRegression_classifier, save_classifier)
save_classifier.close()

In [30]:
SGDClassifier_classifier = SklearnClassifier(SGDClassifier())
SGDClassifier_classifier.train(training_set)
print("SGDClassifier_classifier accuracy percent:", (nltk.classify.accuracy(SGDClassifier_classifier, testing_set))*100)

C:\Users\vivid\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\linear_model\stochastic_gradient.py:128: FutureWarning: max_iter and tol parameters have been added in <class 'sklearn.linear_model.stochastic_gradient.SGDClassifier'> in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
  "and default tol will be 1e-3." % type(self), FutureWarning)


SGDClassifier_classifier accuracy percent: 72.7409638554217


In [31]:
save_classifier = open("pickled_algos/SGDClassifier_classifier5k.pickle","wb")
pickle.dump(SGDClassifier_classifier, save_classifier)
save_classifier.close()

In [32]:
SVC_classifier = SklearnClassifier(SVC())
SVC_classifier.train(training_set)
print("SVC_classifier accuracy percent:", (nltk.classify.accuracy(SVC_classifier, testing_set))*100)

SVC_classifier accuracy percent: 48.493975903614455


In [33]:
save_classifier = open("pickled_algos/SVC_classifier5k.pickle","wb")
pickle.dump(SVC_classifier, save_classifier)
save_classifier.close()

In [38]:
LinearSVC_classifier = SklearnClassifier(LinearSVC())
LinearSVC_classifier.train(training_set)
print("LinearSVC_classifier accuracy percent:", (nltk.classify.accuracy(LinearSVC_classifier, testing_set))*100)

LinearSVC_classifier accuracy percent: 70.48192771084338


In [39]:
save_classifier = open("pickled_algos/LinearSVC_classifier5k.pickle","wb")
pickle.dump(LinearSVC_classifier, save_classifier)
save_classifier.close()

In [40]:
NuSVC_classifier = SklearnClassifier(NuSVC())
NuSVC_classifier.train(training_set)
print("NuSVC_classifier accuracy percent:", (nltk.classify.accuracy(NuSVC_classifier, testing_set))*100)

NuSVC_classifier accuracy percent: 70.03012048192771


In [41]:
save_classifier = open("pickled_algos/NuSVC_classifier5k.pickle","wb")
pickle.dump(NuSVC_classifier, save_classifier)
save_classifier.close()

In [42]:
voted_classifier = VoteClassifier(classifier,
                                  MNB_classifier,
                                  LogisticRegression_classifier,
                                  SGDClassifier_classifier,
                                  LinearSVC_classifier,
                                  SVC_classifier, 
                                  NuSVC_classifier)

In [43]:
print("Voted classifier accuracy percent:", (nltk.classify.accuracy(voted_classifier, testing_set))*100)
print("Classification:", voted_classifier.classify(testing_set[0][0]), "Confidence %:",voted_classifier.confidence(testing_set[0][0])) 
print("Classification:", voted_classifier.classify(testing_set[1][0]), "Confidence %:",voted_classifier.confidence(testing_set[1][0]))
print("Classification:", voted_classifier.classify(testing_set[2][0]), "Confidence %:",voted_classifier.confidence(testing_set[2][0]))
print("Classification:", voted_classifier.classify(testing_set[3][0]), "Confidence %:",voted_classifier.confidence(testing_set[3][0]))
print("Classification:", voted_classifier.classify(testing_set[4][0]), "Confidence %:",voted_classifier.confidence(testing_set[4][0]))

Voted classifier accuracy percent: 72.59036144578313
Classification: neg Confidence %: 1.0
Classification: neg Confidence %: 1.0
Classification: pos Confidence %: 0.8571428571428571
Classification: pos Confidence %: 0.8571428571428571
Classification: neg Confidence %: 1.0
